In [ ]:
import torch
import pandas as pd 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# ✅ 데이터 준비 



# ✅ 하이퍼파라미터 설정
input_size = x_train.shape[2]
hidden_size = 48
dense_size = 64
dropout_rate = 0.4
learning_rate = 0.01
epochs = 150
batch_size = 32

# ✅ 1. PyTorch 모델 정의
class StockLSTM(nn.Module):
    def __init__(self):
        super(StockLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.dense1 = nn.Linear(hidden_size, dense_size)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)
        self.output = nn.Linear(dense_size, 1)  # 회귀이므로 1개 출력

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_output = lstm_out[:, -1, :]  # 마지막 시점의 hidden state
        x = self.dropout1(last_output)
        x = self.relu(self.dense1(x))
        x = self.dropout2(x)
        out = self.output(x)
        return out

# ✅ 2. 데이터 준비 (Tensor로 변환)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)

train_loader = DataLoader(TensorDataset(x_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val_tensor, y_val_tensor), batch_size=batch_size)

# ✅ 3. 학습 준비
model = StockLSTM().to(device)
criterion = nn.L1Loss()  # MAE (mean absolute error)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ✅ 4. 학습 루프
best_val_loss = float('inf')
early_stop_counter = 0
patience = 10  # early stopping patience

for epoch in range(epochs):
    model.train()
    running_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            preds = model(xb)
            val_loss += criterion(preds, yb).item()
    val_loss /= len(val_loader)

    print(f"[Epoch {epoch+1}] Train Loss: {running_loss:.4f}, Val Loss: {val_loss:.4f}")

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
        best_model_state = model.state_dict()
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("⏹ Early stopping triggered.")
            break

# ✅ 5. 최종 모델 저장 or 로드
model.load_state_dict(best_model_state)  # 가장 좋은 상태로 복원
# torch.save(model.state_dict(), "best_lstm_stock_model.pth")